In [ ]:
# Classificação pelo RN

In [1]:
import os
os.chdir('/mnt/mint_files/TCC/main_code/')

In [2]:
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from glob import glob
import rasterio as rio
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

2023-05-22 02:00:44.118755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-22 02:00:44.118772: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Lê a imagem de treino e o seu gabarito

# Assign file names
mxBangalore = 'images/bangalore_data/l5_Bangalore2011_raw.tif'
builtupBangalore = 'images/bangalore_data/l5_Bangalore2011_builtup.tif'
mxHyderabad = 'images/bangalore_data/l5_Hyderabad2011_raw.tif'

# Read the rasters as array
ds1, featuresBangalore = raster.read(mxBangalore, bands='all')
ds2, labelBangalore = raster.read(builtupBangalore, bands=1)
ds3, featuresHyderabad = raster.read(mxHyderabad, bands='all')

# Print the size of the arrays
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (6, 2054, 2044)
Bangalore Binary built-up image shape:  (2054, 2044)
Hyderabad Multispectral image shape:  (6, 1318, 1056)


In [75]:
# Lê as imagens do teste

recreioFile = 'images/lagoa_freitas/2015/lagoa_freitas_v1_B[1-6].TIF'
# recreioFile = 'images/barra/2015/barra_B[1-6].TIF'
recreio_bands = glob(recreioFile)
recreio_bands.sort()

#recreioFile = [
#    'images/barra/2015/barra_B4.TIF',
#    'images/barra/2015/barra_B3.TIF',
#    'images/barra/2015/barra_B2.TIF',
#]
#recreio_bands = recreioFile

l = []
for i in recreio_bands:
    with rio.open(i, 'r') as f:
        l.append(f.read(1))

# Data
arr_st = np.stack(l)

txt = 'Imagem shape D(Bands: {0}, Height: {1}, Width: {2})'.format(arr_st.shape[0], arr_st.shape[1], arr_st.shape[2])
print(txt)

Imagem shape D(Bands: 6, Height: 300, Width: 650)


In [3]:
# Acerta os shape das imagens de treino e de gabarito

# Clean the labelled data to replace NoData values by zero
labelBangalore = (labelBangalore == 1).astype(int)

# Reshape the array to single dimensional array
featuresBangalore = changeDimension(featuresBangalore)
labelBangalore = changeDimension (labelBangalore)
featuresHyderabad = changeDimension(featuresHyderabad)
nBands = featuresBangalore.shape[1]

print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

NameError: name 'labelBangalore' is not defined

In [76]:
# reshape as bandas da imagem de teste
x = np.moveaxis(arr_st, 0, -1)
x.reshape(-1, arr_st.shape[0]).shape, arr_st.shape[1] * arr_st.shape[2]
X_data = x.reshape(-1, arr_st.shape[0])
scaler = StandardScaler().fit(X_data)
X_scaled = scaler.transform(X_data)

X_scaled.shape

(195000, 6)

In [7]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresBangalore, labelBangalore, test_size=0.4, random_state=0)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

(2519025, 6)
(2519025,)
(1679351, 6)
(1679351,)


In [8]:
# Normalise the data
xTrain = xTrain / 255.0 # bits
xTest = xTest / 255.0
featuresHyderabad = featuresHyderabad / 255.0

# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresHyderabad = featuresHyderabad.reshape((featuresHyderabad.shape[0], 1, featuresHyderabad.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresHyderabad.shape)

(2519025, 1, 6) (1679351, 1, 6) (1391808, 1, 6)


In [77]:
# Normaliza a imagem de teste
X_scaled = X_scaled / 65535.0 # 16 bits

# reshape
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
print(X_scaled.shape)

(195000, 1, 6)


In [78]:
# Escolhe as bandas para serem utilizadas. entre 0 e 5. alterar a quantidade na primeira camada da RN
# [1,2,3] -> bands 2,3,4 - RGB
# [2,3,4] -> bands 3,4,5 - Infra vermelho
# [1,4,5] -> bands 2,5,6 - Agricultura
# [0,2,3] -> bands 1,3,4 - Bathymetric
xTrain_b = xTrain[:,:,[1,2,3]]
xTest_b = xTest[:,:,[1,2,3]]
#xTrain_b = xTrain
#xTest_b = xTest

featuresRecreio_b = X_scaled[:,:,[1,2,3]]
#featuresRecreio_b = X_scaled
xTrain_b.shape

(2519025, 1, 3)

In [41]:
# Treinamento da rede

# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 3)), #input_shape=(1, nBands)
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')]) #softmax

# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain_b, yTrain, epochs=2) # testar auamentando o número de epochs

Epoch 1/2
78720/78720 [==============================] - 50s 632us/step - loss: 0.1465 - accuracy: 0.9464
Epoch 2/2
78720/78720 [==============================] - 50s 641us/step - loss: 0.1336 - accuracy: 0.9520


In [79]:
# PRedição da imagem teste

recreioPredicted = model.predict(featuresRecreio_b)
recreioPredicted

array([[0.9594823 , 0.0405177 ],
       [0.9594823 , 0.04051771],
       [0.9594823 , 0.04051773],
       ...,
       [0.9594826 , 0.04051737],
       [0.9594826 , 0.04051738],
       [0.9594826 , 0.04051739]], dtype=float32)

In [53]:
# Pega a pocentagem de cada pixel do resuldado da predição
predictVisualizacao = recreioPredicted[:,1]
predictVisualizacao

array([0.9594827 , 0.9594827 , 0.9594827 , ..., 0.95948195, 0.95948195,
       0.95948195], dtype=float32)

In [54]:
# Cálcula a média das porcentagem
# Usado como possível número de corte

minimo = min(predictVisualizacao)
maximo = max(predictVisualizacao)
media = (minimo + maximo) / 2.0
print(minimo)
print(maximo)
print(media)

0.9594745
0.9594887
0.9594815969467163


In [73]:
# pega o valor de corte e transforma em 1 se a porcentagem for maior ou 0 se for menor ou igual
# Calculate and display the error metrics

valor_corte = media
resultPredict = (predictVisualizacao>valor_corte).astype(int)
resultPredict

array([1, 1, 1, ..., 1, 1, 1])

In [74]:
# Salva em uma imagem em preto e branco

from PIL import Image as im
print(resultPredict)
print(resultPredict.shape)
arrayOfnumeros = np.array(resultPredict)
# (360, 500) - lagoa v1
# (200, 150) - lagoa v2
# (300, 650) - barra
arrayOfnumeros = np.reshape(arrayOfnumeros, (300, 650))
data = im.fromarray((arrayOfnumeros*255).astype(np.uint8))
data.save('images/output/rn/barra/b234/result_rn_b234_0_95909_inv.png')

# plt.matshow(resultPredict);
# plt.colorbar()
# plt.savefig('images/output/rn/result_rn_ball_50_zmth.png')

[1 1 1 ... 1 1 1]
(195000,)
